In [1]:
#importing necessary libraries
import requests
import json
import pandas as pd
import datetime
from decouple import config

In [2]:
def data_validation(df):
    #checking if the dataframe is empty
    if df.empty:
        print('No songs were downloaded')

    #checking if there are duplicates
    if pd.Series(df['played_at']).is_unique:
        pass
    else:
        raise Exception("Primary Key check violated")

    #checking for nulls
    if df.isnull().values.any():
        raise Exception("Found Null values")
    
    return True